# Project Summary: Action Recognition using Recurrent Networks

In computer vision, action recognition refers to the act of classifying an action that is present in a given video and action detection involves locating actions of interest in space and/or time.

Action recognition is a good test environment for Recurrent Networks, since we can consider each frame of a video as sample in the time domain. Also, CNNs will be an important part of our networks since we are not going to feed the raw samples to a recurrent layer, but the samples will be processed by many convolutional layers in order to have a latent representation of the image that we can feed to the RNN.

# 0. Introduction 
Recurrent neural networks, or RNNs, are a family of neural networks for processing sequential data. While convolutional neural network are specialized to process grid of values, such as images, RNNs are used to process sequence of values.
As well as CNNs share the same paramters across all the values, RNNs share the same weights across several time steps. This is how a RNN typically look like:
<img src="images/rnn-unfolded.jpg" width=400px height=200px>
By unrolling/unfolding we mean that the network is written out for the complete sentence. 

The hidden state is the memory of the network and it is usually computed by means of a non-linear functions, such as tanh, and it is calulated based on the current input and the previous state. The output is computed based on the current state.


Training a RNN is similar to training a traditional Neural Network. We also use the backpropagation algorithm, but with a little twist. Because the parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time steps. For example, in order to calculate the gradient at t=4 we would need to backpropagate 3 steps and sum up the gradients. This is called Backpropagation Through Time (BPTT).

## 0.1 Vanilla RNN
In a Vanilla Recurrent Network the repeating module has a very simple structure, such as a single *tanh* layer. The current state in then computed as a function of the previous state $h_{t-1}$and the current input $x_t$.

$$h_t = tanh(W_t*h_{t-1}+W_x*x_t)$$

<img src="images/SimpleRNN.png" width=400px height=200px>

Once obtained the current state is we can compute the output state as 

$$y_t = W_y*h_t$$

### 0.1.1 Gradient problem

Simple (Vanilla) RNNs have difficulties learning long-term dependencies due to what is called the vanishing/exploding gradient problem. 
To understand this problem we must quickly recap what is going on during the training. A loss (error) function L is defined, for example cross-entropy, and computed based on targets and prediction. Then gradients of the error are computed to perform the weight update, that is the **backpropagation algorithm**. 
Gradients must be computed for each weights matrix but, using the chain rule of differentiation, we can notice that to reach the end of the graph we must backpropagate through all the timesteps (as showed in the image below). This is called **backpropagation through time**, basically a fancy name for standard BPP through an unrolled RNN.
<img src="images/bptt.png" width=400px height=200px>

1. **Exploding Gradient**: In recurrent neural networks (or deep networks in general), error gradients can accumulate and become very large numbers resulting in big updates to the network weights. This may eventually bring instability to the network. At an extreme, the values of weights can become so large as to overflow and result in NaN values. The explosion occurs by repeatedly multiplying gradients through the network layers that have values larger than 1.0. In recurrent neural networks, exploding gradients can result in a network that is completely unable to learn from training data and at best a network that cannot learn over long input sequences of data. This problem is usually solved using **gradient clipping**, limiting the norm of the gradients to a certain value.

2. **Vanishing Gradient**: It is the opposite problem but it is following the same mechanism. If the gradient values are small the final gradient will shrink exponentially fast, eventually vanishing completely after a few time steps. Gradient contributions from farther steps become zero, basically the state at those moments does not contribute to what the network is learning, long-term dependencies are not learned. Vanishing gradients also happen in deep Feedforward Neural Networks but in RNNs the problem is more common since they tend to be deeper.

Both the problems are deeply discussed in this paper (http://proceedings.mlr.press/v28/pascanu13.pdf)

## 0.2 LSTM: Long Short-Term Memory
Long Short Term Memory networks are a particular type of RNN, capable of learning long-term dependencies. They are explicitly designed to avoid the long-term dependency problem. Storing information for long periods of time is their default behavior. Consequently LSTM are now extensively used on a large assortment of problems.
LSTMs have the same chain structure as vannilla RNNs but the repeating module has a more complex structure, instead of one layer there are four interacting in a specific way.

<img src="images/LSTM3.png" width=400px height=200px>



The key to LSTMs is the **cell state**, computed following the horizontal line running through the top of the diagram. The cell state is one of the two hidden states that are passed through time. 
The LSTM has the possibility to remove or add information to the cell state, regulated by structures called gates.
Gates are usually composed of a sigmoid neural net layer and a pointwise multiplication operation. There are three of these gates in a standartd LSTM cell:
* **Forget Gate**: takes $h_{t-1}$ and $x_t$ as input and outputs a number between 0 and 1 for each component of the cell state $C_{t−1}$. A 1 keeps the value while a 0 forgets it. 

$$f_t = \sigma(W_f*[h_{t-1},x_t] + b_f)$$
* **Input Gate**: used to update the values of the cell state, combined to a vector of new candidate values $\tilde{C_t}$.

$$i_t = \sigma(W_i*[h_{t-1},x_t]+b_i)$$ $$\tilde{C_t}= tanh(W_C*[h_{t-1},x_t]+b_C)$$
* **Output Gate**: in the same way as the other gates decides what the network is going to output. It will be then combined with the current state. 

$$o_t = \sigma(W_o*[h_{t-1},x_t]+b_o)$$



The new **Cell State** $C_t$ and the new hidden state $h_y$ are computed as follows

$$C_t=f_t*C_{t-1}+i_t*\tilde{C_t}$$
$$h_t=o_t*tanh(C_t)$$

LSTMs and variation of this concept solve the vanishing gradient problem by eliminating the activation function for the cell state. Therefore the activation function is the identity which leads to a gradient of 1. Since you still want to manipulate the hidden state, input, output and forget gate are included in an LSTM unit.

### 0.3 Experiments on simple dataset for sequence analysis (IMDB dataset)

First we performed some simple experiments on the IMDB dataset, available on Keras. We tried different recurrent layers (Vanilla RNNs, LSTM, Bidirectional LSTM, Embedding layers) in order to have a little of familiarity with the code.

# 1. Information about our datasets and data preparation

### 1.1 Our datasets

Our test datasets come from the University of California and are used as benchmarks for many action recognition works. We are talking about [UCF11](http://crcv.ucf.edu/data/UCF_YouTube_Action.php) (also known as YouTube Action Dataset), [UCF50](http://crcv.ucf.edu/data/UCF50.php) and [UCF101](http://crcv.ucf.edu/data/UCF101.php). These 3 datasets contain raw videos that mostly involve human actions (sports, playing music, ...). The resolution is usually 320x240 and the length of the videos changes significantly from video to video, but on average they contain 200 frames. The number of different actions changes with the dataset (11 for UCF11, 50 for UCF50, 101 for UCF101) and the number of videos is almost constant for each class, so UCF101 ends up to be 10 times bigger than UCF11. Also, for each class/action the videos are divided into subgroups of videos that have probably been extrapolated from the same longer video.
The structure of the dataset is therefore the following:

* class_1
    * subgroup_1
        * video_1_1
        * video_1_2
        * ...
        * video_1_n
    * subgroup_2
        * video_2_1
        * video_2_2
        * ...
        * video_2_n
    * ....
    * subgroup_m
* ...
* class_k


We also tested another well-known dataset for action recognition, [HMDB51](http://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database). This is a harder dataset and contains videos about human activities.

### 1.2 Preprocessing of the datasets

In order to train our models, we need to extrapolate the frames from our videos. We decided to implement the extrapolation and saving of the frames as a preprocessing step in order to avoid repeating this operation each time we train a different model. Also, in order to access the frame at index $i$, all the $i-1$ frames have to be scanned, and this would be highly inefficient when training.

The preprocessing step uses the OpenCV VideoCapture API to open every video of a dataset sequentially, extracts a **fixed** number of frames and saves the frames as JPEG files (the output filename contains an incremental index, allowing to order the images during the loading phase). It is also possible to specify a different output format, that involves feeding a neural network (such as InceptionV3 or Yolo V2, see below) the frames and save the output as a npy file.

As already mentioned the number of extracted/preprocessed frames is fixed, because a RNN requires sequences with fixed length. The number of frames that we used is therefore much lower than the average number of frames per video; we used values between 30 and 50. The extrapolated frames are not adjacent, but they are sampled at a fixed step:

$$ sampling\_step = \frac{total\_number\_of\_video\_frames}{number\_of\_frames\_to\_be\_extracted} $$

If a video has a lot of frames, we are therefore going to extract a fast-paced version of the video. If a video doesn't have enough frames (less than the number of frames that we want to extract), it is rejected.

The preprocessing step also splits the dataset into training and test set (we chose not to split the dataset into train/validation/test set because it didn't seem useful for our experiments). It is possible to specify the percentage of videos used for the training set. It is worth mentioning that videos that belong to the same subgroup will not be splitted into training and test set; either all of them will be used for training, or all of them will be used for testing. It would be useless to train a neural network otherwise, because the network would simply recognize frames captured from the same macro-video inside the test set.

# 2. Use a CNN to classify each frame individually

In our first approach we didn't use recurrent networks; instead, we tried to classify each frame separately. In order to classify an entire video, we computed the mean prediction on all the video frames.

### 2.1 Train a convolutional network from scratch
 
In our first experiment we trained a convolutional neural network with randomly initialized weights. We tried different configurations of the network but we were never able to let the network learn anything. This could be related to the fact that in a video there could be a mixture of "meaningful" frames, where we can easily identify the action, and "useless" frames, were we don't have enough information to know what's happening (e.g. missing objects or people in the scene). Also, with many frames coming from the same video, it's hard to foresee a network that, even if it works, will not overtrain.

Anyway, the accuracy of the network on the training set was never higher than $100/number\_of\_classes$, meaning that the network was guessing randomly.

### 2.2 Use a pre-trained CNN (InceptionV3)

Since training a CNN from scratch didn't seem to be possible, we decided to switch to a pre-trained and well-known model: InceptionV3. The model, available on Keras, has been trained on the ImageNet dataset and has 0.78 top-1 accuracy. This is a very *deep* network: 159 and 24M weights! This is why we decided to save the output of this model in our preprocessing phase, so that we didn't have to feed the model with our frames when training. Also, we didn't take the output of the last layer (the 1000 predictions of the ImageNet dataset), but we took the output of the last convolutional layer, that most certainly contains more meaningful information. To save storage space and training time, we also decided to apply a global average pooling to the convolutional output: the output of the InceptionV3 network, that becomes the input for our custom networks, is therefore a 1D vector of size 2048.

Since the input vector is 1D and the images have been preprocessed by a lot of convolutional layers, our custom network becomes very simple: a fully connected layers (256 to 512 weight) and a second fully connected layers that outputs the predictions, with some dropout in between. This very simple model turned out to be very fast and accurate, yielding the following results:

| Dataset        | Top-1 Accuracy           | Top-3 Accuracy  |
| ------------- |-------------:| -----:|
| UCF11       | 88.9           | 98.7 |
| UCF50       | 82.1      |   92.6 |
| UCF101      | 79.0      |    90.4 |
| HMDB51      | 46.2      | 69.3    |


### 2.2.1 How to compute the video class given the individual frames predictions

Since our network outputs one result for each frame, we tried different methods to compute the class of a video given its individual frames predictions:

1. Compute the mean of the predictions, and then taking the argmax
2. Create a counter of length == number_of_classes. For each frame prediction, take the top-1 prediction and increase the relative counter. Take the class with the highest counter

These two methods revealed to yield very similar results, with the first approach being usually slightly better than the second (the difference between the two results is always less than 1%).

# 3. Use a RNN to classify sequences of frames

This second approach consists in classifying an entire video by enforcing the network to evaluate the dependencies between consecutive frames with a LSTM layer. Since training a network from scratch was very problematic, we spent most of our efforts into training a recurrent network using as input the 1D output vector of Inception. Our network takes an input that is defined as a 2D vector with dimensions (NB\_FRAMES, INCEPTION\_FEATURES) (e.g. (50, 2048)).

The networks that we tested still contain a few layers (just LSTM and fully connected layers), but using a recurrent layer resulted in much more weights (magnitude of 10^6 or even 10^7) and in a slower training.

We explored different configurations of the hyperparameters. The basic network that we used for testing contains one LSTM layer, one intermediate fully connected layers, and the last fully connected layer with the output predictions.
* Changing the output size of the LSTM: a value that is too small (e.g. 32) doesn't allow the network to learn anything. When this number is big enough, the network starts to learn. Increasing this value after a certain limit doesn't help the network to have a better accuracy.
* Change learning rate and optimizer: we switched from the adam optimizer to sgd+momentum+nesterov with a slower learning rate. Using a slow learning rate allowed the network to reach slightly higher test accuracies.
* Use a stack of LSTM layers: we tried to stack 2 or 3 LSTM layers, hoping that the network would learn higher-level dependencies. Unfortunately, the stacked-LSTM networks were more prone to overfitting and the 1-LSTM-layer network proved to be always the best one.
* Play with recurrent dropout, regularizers and norm clipping: they seemed to help the network to avoid overfitting in the first epochs, but after many epochs the training accuracy always became much higher than the test accuracy.
* Decrease the learning rate over time when the test accuracy wasn't increasing for n consecutive epochs: again, no improvement.
* Use a variation of the LSTM layer that returns one output for each timestep; we then added a layer that computes the mean of the output sequence. 
* Use a bidirectional LSTM, which learns dependencies by analazying the sequence in both directions (from "past to future" and "from future to past"). We achieved the same results of a non-bidirectional LSTM.
* Train with more frames per video, by padding with black frames (all zeros) at the beginning of a video: this is a technique that is used with variable length sequences, such as text sequences. By using padding, we were able to increase the number of extracted frames, hoping that it would help the RNN to recognize the action better.
* We also tried to train a RNN using as input the frame predictions of the non-RNN approach. Counterproductive.

After all these tests, <u>we weren't able to find a recurrent network that outperforms by far the non-RNN approach</u>. We always achieved more or less the same results: only on the HMDB51 we were able to achieve 48.4% accuracy using a bidirectional LSTM (+2% accuracy compared to the non-RNN apprach).

The reason why the accuracy didn't increase may be related to the input data: the Inception model does not know anything about sequences and it's therefore possible that its "answer" to similar frames of the same video is the same. The RNN is then trying to learn temporal dependencies between latent vectors that "mean" the same thing. We also observed that with the non-RNN approach some sequences are wrongly classified and the best predicted (wrong) class has a probability near to 1. This is a confirmation that some videos are misinterpreted by Inception and therefore we cannot expect that a RNN will learn a better classification.

# 4. Attempt to use Yolo v2 object detections for temporal learning

[YOLO](https://pjreddie.com/darknet/yolo) (You Only Look Once) is a very popular convolutional network for real time object detection. Since its output for sure will be different for different frames of a video (since objects/people move in the scene), we could use its output for our RNN training. 

We used [darkflow](https://github.com/thtrieu/darkflow), a TensorFlow implementation of Yolo, and the pre-trained model on the COCO dataset (80 classes). The output of Yolo, for one image, is a tensor for size 19x19x425; we can think of it as a grid of size 19x19 where for each cell we have the predictions about 5 bounding boxes and their relative class (425/5 = 85, 80 values for the onehot classes, 5 values for the coordinates of the bounding box).

Since the output of Yolo is a 3D tensor, we can consider that as an image; in fact, in our first approach we trained a convolutional network with a recurrent layer on the Yolo output and we were able to obtain 70% accuracy on UCF11.

We then built a neural network that concatenates Yolo+CNN and Inception outputs, sends the sequence of concatenated vectors to a RNN, and predicts the class of the video. 

<img src="images/costa.jpg" width=400px height=200px>

In this case we were able to obtain a small improvement on UCF11 (91% top-1 accuracy). We weren't able to perform further experiments because of time (cost of the virtual machine, very slow training time) and memory (we needed a lot more RAM to store Yolo results for bigger datasets, thus allowing for a faster training) costraints.

<hr/>

The architecture that we used:
* Hardware: virtual machine on Google Cloud
    * 8 vCPU
    * 52 GB RAM
    * 500 GB SSD
    * Nvidia Tesla P100 (16 GB VRAM)
* OS: Ubuntu 16.04 LTS
* Software:
    * CUDA 9.0 with CuDNN
    * Python 3.5.2
    * TensorFlow 1.5
    * Keras 2.1.4